<a href="https://colab.research.google.com/github/vahidseydi/Regression/blob/master/Multiple_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv('kc_house_data.csv')
data.head()

FileNotFoundError: ignored

In [0]:
Y=data['price']
X=data
X=X.drop(['price'],axis=1)
X.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [0]:
features=['bedrooms','bathrooms','sqft_living','sqft_lot','floors','yr_built']
H=X[features]
H['intersept']=1
H.head()

/home/vahid/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,bedrooms,bathrooms,sqft_living,sqft_lot,floors,yr_built,intersept
0,3,1.00,1180,5650,1.0,1955,1
1,3,2.25,2570,7242,2.0,1951,1
2,2,1.00,770,10000,1.0,1933,1
3,4,3.00,1960,5000,1.0,1965,1
4,3,2.00,1680,8080,1.0,1987,1


In [0]:
d=X.shape[1]
D=H.shape[1]
N=len(H)
print('d=%d,D=%d,N=%d'%(d,D,N))

d=20,D=7,N=21613


In [0]:
H=np.matrix(H)


In [0]:
HT=H.T
HT.shape

(7, 21613)

In [0]:
H.shape

(21613, 7)

In [0]:
temp1=HT*H

In [0]:
temp1.shape

(7, 7)

In [0]:
temp2=temp1.I
temp2

matrix([[ 8.40242254e-05, -2.76454855e-05, -3.69800177e-08,
          1.19545359e-10,  1.08698346e-05,  2.17629546e-07,
         -5.94851946e-04],
        [-2.76454855e-05,  2.45493089e-04, -1.12559711e-07,
          1.01821580e-10, -5.99930351e-05, -1.47075569e-06,
          2.79512009e-03],
        [-3.69800177e-08, -1.12559711e-07,  1.48969106e-10,
         -3.71425058e-13, -5.45020892e-09,  2.69316515e-10,
         -4.64219381e-07],
        [ 1.19545359e-10,  1.01821580e-10, -3.71425058e-13,
          2.82274157e-14,  1.78091828e-10, -1.95568717e-12,
          3.31634505e-09],
        [ 1.08698346e-05, -5.99930351e-05, -5.45020892e-09,
          1.78091828e-10,  2.37624680e-04, -1.35376563e-06,
          2.41207000e-03],
        [ 2.17629546e-07, -1.47075569e-06,  2.69316515e-10,
         -1.95568717e-12, -1.35376563e-06,  8.17255463e-08,
         -1.57212438e-04],
        [-5.94851946e-04,  2.79512009e-03, -4.64219381e-07,
          3.31634505e-09,  2.41207000e-03, -1.57212438e-04

In [0]:
np.matrix

Temp1=H*HT
Temp2=np.invert(temp1)
temp3=Temp2*HT
w=temp3*Y

